# Dockerfile

## Commands

### FROM

- baseimages

### MAINTAINER

### ADD

In [ ]:
primeri coursera





**Connect to the nginx using the internal IP**

Get the internal IP address either copying from the full inspect file or by assigning it to a shell variable:

    CN="sharp_bartik"
    CIP=$(sudo docker inspect --format '{{ .NetworkSettings.IPAddress }}' $CN)
    curl  http://$CIP

You can also get all instance IDs and their corresponding IP addresses by doing this:

    sudo docker inspect -f '{{.Name}} - {{.NetworkSettings.IPAddress }}' $(sudo docker ps -aq)

**Stop an instance**

    sudo docker stop <cid>
    
or sudo docker stop $(sudo docker ps -aq)

**Verify no more instances running**
    
    sudo docker ps

**Remove the docker containers from the system**

    sudo docker rm <cid>
or sudo docker rm $(sudo docker ps -aq)

## Create an image

Commands to run on the VM Instance

Install Go

    wget https://storage.googleapis.com/golang/go1.6.2.linux-amd64.tar.gz
    rm -rf /usr/local/bin/go
    sudo tar -C /usr/local -xzf go1.6.2.linux-amd64.tar.gz
    export PATH=$PATH:/usr/local/go/bin
    export GOPATH=~/go

Get the app code

    mkdir -p $GOPATH/src/github.com/udacity
    cd $GOPATH/src/github.com/udacity
    git clone https://github.com/udacity/ud615.git

Build a static binary of the monolith app

    cd ud615/app/monolith
    go get -u
    go build --tags netgo --ldflags '-extldflags "-lm -lstdc++ -static"'

Why did you have to build the binary with such an ugly command line?

You have to explicitly make the binary static. This is really important in the Docker community right now because alpine has a different implementation of libc. So your go binary wouldn't have had the lib it needed if it wasn't static. You created a static binary so that your application could be self-contained.

**Create a container for the app**

Look at the Dockerfile

    cat Dockerfile
    
Build the app container

    sudo docker build -t monolith:1.0.0 .
    
List the monolith image

    sudo docker images monolith:1.0.0

Run the monolith container and get it's IP

    sudo docker run -d monolith:1.0.0
    sudo docker inspect <container name or cid>
    
or

    CID=$(sudo docker run -d monolith:1.0.0)
    CIP=$(sudo docker inspect --format '{{ .NetworkSettings.IPAddress }}' ${CID})

Test the container
    curl <the container IP>
    
or

    curl $CIP

Important note on security

If you are tired of typing "sudo" in front of all Docker commands, and confused why a lot of examples don't have that, please read the following article about implications on security - Why we don't let non-root users run Docker in CentOS, Fedora, or RHEL

http://www.projectatomic.io/blog/2015/08/why-we-dont-let-non-root-users-run-docker-in-centos-fedora-or-rhel/

Create docker images for the remaining microservices - auth and hello.
Repeat the steps you took for monolith.

Build the auth app

    cd $GOPATH/src/github.com/udacity/ud615/app
    cd auth
    go build --tags netgo --ldflags '-extldflags "-lm -lstdc++ -static"'
    sudo docker build -t auth:1.0.0 .
    CID2=$(sudo docker run -d auth:1.0.0)
    
Build the hello app

    cd $GOPATH/src/github.com/udacity/ud615/app
    cd hello
    go build --tags netgo --ldflags '-extldflags "-lm -lstdc++ -static"'
    sudo docker build -t hello:1.0.0 .
    CID3=$(sudo docker run -d hello:1.0.0)
    
See the running containers

    sudo docker ps -a

In [ ]:
   sudo dpkg -l | grep nginx

## Docker registers

See all images

    sudo docker images

Docker tag command help

    docker tag -h

    
Add your own tag
    
    sudo docker tag monolith:1.0.0 <your username>/monolith:1.0.0

For example (you can rename too!)
    
    sudo docker tag monolith:1.0.0 udacity/example-monolith:1.0.0

            
Create account on Dockerhub

To be able to push images to Dockerhub you need to create an account there - https://hub.docker.com/register/

Login and use the docker push command

    sudo docker login
    sudo docker push udacity/example-monolith:1.0.0

Repeat for all images you created - monolith, auth and hello!